In [1]:
# TF CODE
import tensorflow as tf
import numpy as np
import random
import os

seed=1234
def seed_everything(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['pythonhashseed'] = str(seed)
    tf.random.set_random_seed(seed)
seed_everything(seed)

# Load data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# Create the model
x = tf.placeholder(tf.float32, [None, 784], name="x-input")
W = tf.Variable(tf.zeros([784,10]), name="weights")
b = tf.Variable(tf.zeros([10], name="bias"))

# use a name scope to organize nodes in the graph visualizer
with tf.name_scope("Wx_b") as scope:
  y = tf.nn.softmax(tf.matmul(x,W) + b)

# Add summary ops to collect data
w_hist = tf.summary.histogram("weights", W)
b_hist = tf.summary.histogram("biases", b)
y_hist = tf.summary.histogram("y", y)

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10], name="y-input")
# More name scopes will clean up the graph representation
with tf.name_scope("xent") as scope:
  cross_entropy = -tf.reduce_sum(y_*tf.log(y))
  ce_summ = tf.summary.scalar("cross_entropy", cross_entropy)
with tf.name_scope("train") as scope:
  train_step = tf.train.AdamOptimizer(0.02).minimize(cross_entropy)

with tf.name_scope("test") as scope:
  correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  accuracy_summary = tf.summary.scalar("accuracy", accuracy)

# Merge all the summaries and write them out to /tmp/mnist_lin
merged = tf.summary.merge_all()
sess = tf.Session()
writer = tf.summary.FileWriter("./mnist_lin", sess.graph_def)
init = tf.global_variables_initializer()
sess.run(init)

# Train the model, and feed in test data and record summaries every 10 steps

for i in range(1000):
  if i % 10 == 0:  # Record summary data, and the accuracy
    feed = {x: mnist.test.images, y_: mnist.test.labels}
    result = sess.run([merged, accuracy], feed_dict=feed)
    summary_str = result[0]
    acc = result[1]
    writer.add_summary(summary_str, i)
    print("Accuracy at step %s: %s" % (i, acc))
  else:
    batch_xs, batch_ys = mnist.train.next_batch(100)
    feed = {x: batch_xs, y_: batch_ys}
    sess.run(train_step, feed_dict=feed)

print(sess.run(accuracy, feed_dict = {x: mnist.test.images, y_: mnist.test.labels}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Accuracy at step 0: 0.098
Accuracy at step 10: 0.7944
Accuracy at step 20: 0.856
Accuracy at step 30: 0.8835
Accuracy at step 40: 0.8902
Accuracy at step 50: 0.8884
Accuracy at step 60: 0.8929
Accuracy at step 70: 0.8997
Accuracy at step 80: 0.9018
Accuracy at step 90: 0.9023
Accuracy at step 100: 0.

In [0]:
from jax import jit, grad, vmap, random
from functools import partial
import jax
import jax.numpy as np
from jax.experimental import stax # neural network library
from jax.experimental.stax import Conv, Dense, MaxPool, Relu, Flatten, LogSoftmax, LeakyRelu, Dropout # neural network layers
from jax.nn import softmax
from jax.nn.initializers import zeros
import matplotlib.pyplot as plt # visualization
import numpy as onp
from jax.experimental import optimizers
from jax.tree_util import tree_multimap  # Element-wise manipulation of collections of numpy arrays

In [3]:
# Load data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
rng = random.PRNGKey(1)
print(mnist.train.images.shape, mnist.train.labels.shape, mnist.test.images.shape, mnist.test.labels.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10) (10000, 784) (10000, 10)


In [4]:
#Define model
in_dim = mnist.train.images.shape[-1]
out_dim = mnist.train.labels.shape[-1]
_, net_apply = Dense(out_dim)
in_shape = (-1, in_dim)
net_params = np.zeros((in_dim,out_dim)), np.zeros((1,out_dim))
print(net_params)

(_FilledConstant([[0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 ...,
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), _FilledConstant([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32))


In [0]:
#Define losses and optimizers
def loss(params, inputs, targets):
    # Computes average loss for the batch
    predictions = softmax(net_apply(params, inputs), axis=-1)
    print(targets, predictions)
    return -np.sum(targets*np.log(predictions))

def batch_loss(params, inputs, targets):
    losses=vmap(partial(loss, params))(inputs, targets)
    return np.mean(losses)

def accuracy(params, inputs, targets):
    predictions = softmax(net_apply(params, inputs), axis=-1)
    correct_prediction = np.equal(np.argmax(targets,-1), np.argmax(predictions,-1))
    return np.mean(correct_prediction)

opt_init, opt_update, get_params = optimizers.adam(step_size=0.01)  # this LR seems to be better than 1e-2 and 1e-4
# out_shape, net_params = net_init(rng, in_shape)
net_params = np.zeros((in_dim,out_dim)), np.zeros((1,out_dim))
opt_state = opt_init(net_params)

@jit
def step(i, opt_state, x, y):
    p = get_params(opt_state)
    g = grad(batch_loss)(p, x, y)
    l = batch_loss(p, x, y)
    return opt_update(i, g, opt_state), l

In [6]:
#Training
losses=[]
for i in range(1000):
    if i % 10 == 0:
        # print(i, mnist.test.images.shape, mnist.test.labels.shape)
        current_params = get_params(opt_state)
        print("Accuracy at step %s: %s" % (i, accuracy(current_params, mnist.test.images, mnist.test.labels)))
    else:
        batch_xs, batch_ys = mnist.train.next_batch(100)
        opt_state, l = step(i, opt_state, batch_xs, batch_ys)
        losses.append(l)
        #print(l)
net_params=get_params(opt_state)

Accuracy at step 0: 0.098000005
Traced<ShapedArray(float32[10])>with<BatchTrace(level=2/1)> Traced<ShapedArray(float32[1,10])>with<BatchTrace(level=2/1)>
Traced<ShapedArray(float32[10])>with<BatchTrace(level=0/1)> Traced<ShapedArray(float32[1,10])>with<BatchTrace(level=0/1)>
Accuracy at step 10: 0.80660003
Accuracy at step 20: 0.84560007
Accuracy at step 30: 0.86490005
Accuracy at step 40: 0.88080007
Accuracy at step 50: 0.88290006
Accuracy at step 60: 0.89230007
Accuracy at step 70: 0.89330006
Accuracy at step 80: 0.89750004
Accuracy at step 90: 0.89480007
Accuracy at step 100: 0.90080005
Accuracy at step 110: 0.9035
Accuracy at step 120: 0.90250003
Accuracy at step 130: 0.90250003
Accuracy at step 140: 0.9039
Accuracy at step 150: 0.9046
Accuracy at step 160: 0.90760005
Accuracy at step 170: 0.91240007
Accuracy at step 180: 0.90860003
Accuracy at step 190: 0.91470003
Accuracy at step 200: 0.9064
Accuracy at step 210: 0.9096
Accuracy at step 220: 0.9110001
Accuracy at step 230: 0.9153